In [53]:
import geopandas as gpd
import pandas as pd
import numpy as np
from rasterstats import zonal_stats
from osgeo import gdal
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# 1. Chargement des données
shp_path = "../results/data/sample/Sample_BD_foret_T31TCJ.shp"
gdf = gpd.read_file(shp_path)

# Charger le raster avec GDAL
raster_path = "../results/data/classif/carte_essences_echelle_pixel.tif"
raster = gdal.Open(raster_path)

In [92]:
def classif_polygone(stats, area_ha, id, cn):
    total = sum(stats.values())
    print(f"ID: {id}, Statistiques : {stats}, Total pixels : {total}, Surface (ha) : {area_ha}")  # Affichage des stats

    if total == 0:
        return np.nan  # Si total est 0, retourner une valeur NaN

    # Si total n'est pas zéro, procéder avec les calculs
    feuillus = (stats.get(11, 0) + stats.get(12, 0) + stats.get(13, 0) + stats.get(14, 0)) / total * 100  # Pourcentage d'essences propres de feuillus
    coniferes = (stats.get(21, 0) + stats.get(22, 0) + stats.get(23, 0) + stats.get(24, 0) + stats.get(25, 0)) / total * 100  # Pourcentage d'essences propres de conifères
    classes_C = [11, 12, 13, 14, 21, 22, 23, 24, 25] # Définition de la classe C

    if area_ha < 2:  # Surface < 2 ha
        if feuillus > 75:
            return 16.0
        elif coniferes > 75:
            return 27.0
        elif coniferes > feuillus:
            return 28.0
        else:
            return 29.0
    else:  # Surface > 2 ha
        for code in classes_C:
            if stats.get(code, 0) / total * 100 > 75:
                return cn.get(code)
        if feuillus > 75:
            return 15.0
        elif coniferes > 75:
            return 26.0
        elif coniferes > feuillus:
            return 28.0
        else:
            return 29.0

In [93]:
# Extraire les noms associés aux codes dynamiquement depuis Sample_BD_foret_T31TCJ
if "Code_objet" in shp.columns:
    cn = dict(shp["Code_objet"])

# Calculer les statistiques zonales
stats = zonal_stats(shp_path, raster_path, categorical=True, nodata=0)

# Calculer la surface de chaque polygone en hectares
shp["surface_ha"] = shp.geometry.area / 10000  # Conversion m² -> ha

# Assigner les classes de peuplement, en vérifiant la colonne 'Code_pixel'
shp["code_predi"] = [
    classif_polygone(stat, area_ha, i, cn) if pd.isna(shp.loc[idx, "Code_pixel"]) else -1
    for idx, (stat, area_ha, i) in enumerate(zip(stats, shp["surface_ha"], shp["ID"]))
]

# Sauvegarder le shapefile avec les modifications dans le même fichier
shp.to_file(shp_path)

# Afficher un message pour confirmer la mise à jour
print(f"Le shapefile a été mis à jour avec le champ 'code_predi' sous : {shp_path}")


ID: FORESTIE0000000003105185, Statistiques : {12: 68}, Total pixels : 68, Surface (ha) : 0.6888999026935516
ID: FORESTIE0000000003105187, Statistiques : {11: 192, 12: 282}, Total pixels : 474, Surface (ha) : 4.770197219416126
ID: FORESTIE0000000003105189, Statistiques : {11: 2, 12: 11}, Total pixels : 13, Surface (ha) : 0.1401122738619964
ID: FORESTIE0000000003106254, Statistiques : {12: 901}, Total pixels : 901, Surface (ha) : 8.984835973870105
ID: FORESTIE0000000003106355, Statistiques : {12: 561}, Total pixels : 561, Surface (ha) : 5.590644987527783
ID: FORESTIE0000000003106735, Statistiques : {11: 5, 12: 6612, 14: 10, 24: 6}, Total pixels : 6633, Surface (ha) : 66.42809821273995
ID: FORESTIE0000000003106775, Statistiques : {12: 17}, Total pixels : 17, Surface (ha) : 0.17769237696246776
ID: FORESTIE0000000003106776, Statistiques : {12: 319}, Total pixels : 319, Surface (ha) : 3.226766617996828
ID: FORESTIE0000000003106777, Statistiques : {12: 4}, Total pixels : 4, Surface (ha) : 0.0

In [94]:
# Suppression des colonnes 'code_predi', 'code_pre_1', et 'surface_ha'
shp = shp.drop(columns=['surface_ha'])

# Afficher les 5 premières lignes pour vérifier
print(shp.head())

                         ID   CODE_TFV  \
0  FORESTIE0000000003104299  FF1G01-01   
1  FORESTIE0000000003105185     FF1-00   
2  FORESTIE0000000003105187  FF1-00-00   
3  FORESTIE0000000003105189     FF1-00   
4  FORESTIE0000000003105624         FP   

                                      TFV                TFV_G11  \
0     Forêt fermée de chênes décidus purs  Forêt fermée feuillus   
1  Forêt fermée de feuillus purs en îlots  Forêt fermée feuillus   
2      Forêt fermée à mélange de feuillus  Forêt fermée feuillus   
3  Forêt fermée de feuillus purs en îlots  Forêt fermée feuillus   
4                              Peupleraie             Peupleraie   

          ESSENCE   Nom_pixel  Code_pixel            Nom_objet  Code_objet  \
0  Chênes décidus       Chêne        12.0                Chêne        12.0   
1        Feuillus        None         NaN    feuillus en îlots        16.0   
2        Feuillus        None         NaN  Mélange de feuillus        15.0   
3        Feuillus        N

In [98]:
# Calculer la matrice de confusion
shp_path_matrix = shp[shp["code_predi"] != -1]

# Enlever les valeurs NaN
shp_path_matrix = shp_path_matrix.dropna(subset=["Code_objet", "code_predi"])
y_true = shp_path_matrix["Code_objet"]
y_pred = shp_path_matrix["code_predi"]

cm = confusion_matrix(y_true, y_pred)
print("Matrice de confusion :")
print(cm)

unique_labels = sorted(set(y_true) | set(y_pred))

Matrice de confusion :
[[   0    0    0    0    0    0    0]
 [2567   17   47    0    0    1    5]
 [   0    0 5307    0    0    0   15]
 [   4    1    1    1    0   10    2]
 [   0    0    0    0    0    0    0]
 [  29    2   25    1    2   18   42]
 [  84    6   71    0    2    8   28]]
